In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip -q install pdfplumber seaborn sentence-transformers scikit-learn unidecode

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os, re, glob, pdfplumber

# Diretórios principais
BASE = "/content/drive/MyDrive/pcj_analise"
PDF_DIR = f"{BASE}/pdf"
OUT_DIR = f"{BASE}/resultados"
os.makedirs(OUT_DIR, exist_ok=True)

# Caminho da base com valores atuais (na pasta resultados)
PATH_BASE_ATUAIS = f"{OUT_DIR}/df_resumo_normalizado.csv"

# Lê a base confiável com os valores atuais
df_base = pd.read_csv(PATH_BASE_ATUAIS)

print("📄 Base com valores atuais carregada:")
print(df_base.head())


📄 Base com valores atuais carregada:
  mes_ano;rio;chuva;vazao;oxigenio;chuva_norm;vazao_norm;oxigenio_norm
0  2025-01;Rio Piracicaba em Artemis;211.0;151.39...                  
1  2025-01;Rio Piracicaba em Monte Alegre;161.0;1...                  
2  2025-01;Rio Atibaia;110.0;9.62;0.0;0.519;0.052...                  
3  2025-01;Rio Jaguari;193.0;2.16;0.67;0.914;0.00...                  
4  2025-01;Rio Jundiaí;181.0;13.92;44.14;0.857;0....                  


In [ ]:
def basic_clean(text):
    return re.sub(r"[ \t]{2,}", " ", text.replace("\xa0"," ")).strip()

def extract_pdf_text(path):
    pages = []
    with pdfplumber.open(path) as pdf:
        for p in pdf.pages:
            txt = p.extract_text() or ""
            pages.append(basic_clean(txt))
    return pages

def guess_month_year(fn):
    meses = dict(jan='01',fev='02',mar='03',abr='04',mai='05',jun='06',
                 jul='07',ago='08',set='09',out='10',nov='11',dez='12')
    s = fn.lower()
    m = re.search(r'(jan|fev|mar|abr|mai|jun|jul|ago|set|out|nov|dez)[^\d]*(\d{4})', s)
    if m:
        return f"{m.group(2)}-{meses[m.group(1)]}"
    return "desconhecido"


In [ ]:
import re, os

def parse_numbers_from_text(pages, month_tag, file_name):
    """
    Extrai dados dos boletins PCJ (jan–set 2025):
    - Código da estação (ex: EF06)
    - Primeiro par: valor atual (mm / m³/s)
    - Segundo par: média histórica (mm / m³/s)
    - Corrige extração de oxigênio (pega o primeiro % após o bloco de Vazão)
    """
    rows = []

    for pi, page in enumerate(pages, start=1):
        text = re.sub(r'\s+', ' ', page.replace('\n', ' '))

        # Código da estação (ex: EF06, EF31, etc.)
        cod_match = re.search(r'\b(EF\d{1,3})\b', text, flags=re.IGNORECASE)
        if not cod_match:
            continue
        codigo_estacao = cod_match.group(1).upper().strip()

        # Captura pares "X mm Y m³/s"
        pairs = re.findall(r'(\d+(?:[.,]\d+)?)\s*mm\s+(\d+(?:[.,]\d+)?)\s*m[³3]\s*/\s*s', text, flags=re.IGNORECASE)
        if len(pairs) < 2:
            continue

        # Ordem correta: primeiro = atual, segundo = histórico
        chuva, vazao = [float(x.replace(',', '.')) for x in pairs[0]]
        chuva_hist, vazao_hist = [float(x.replace(',', '.')) for x in pairs[1]]

        # Captura oxigênio — o primeiro % depois do bloco de "m³/s"
        oxig_match = re.search(r'(\d+(?:[.,]\d+)?)\s*%\s*(?=Precipitação Vazão Oxigênio)', text, flags=re.IGNORECASE)
        if oxig_match:
            oxig = float(oxig_match.group(1).replace(',', '.'))
        else:
            oxig = None

        rows.append({
            'mes_ano': month_tag,
            'arquivo': os.path.basename(file_name),
            'pagina': pi,
            'codigo_estacao': codigo_estacao,
            'chuva': chuva,
            'chuva_hist': chuva_hist,
            'vazao': vazao,
            'vazao_hist': vazao_hist,
            'oxigenio': oxig
        })

    return rows


In [ ]:
import pandas as pd

# Lista todos os arquivos PDF da pasta
pdf_files = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))

all_rows = []

for pdf in pdf_files:
    pages = extract_pdf_text(pdf)
    month_tag = guess_month_year(os.path.basename(pdf))
    res = parse_numbers_from_text(pages, month_tag, pdf)
    all_rows.extend(res)

    # Gera DataFrame parcial do mês
    df_mes = pd.DataFrame(res)
    print(f"\n📄 {os.path.basename(pdf)} — {len(df_mes)} registros extraídos")

    if not df_mes.empty:
        print(df_mes[['codigo_estacao', 'chuva', 'chuva_hist', 'vazao', 'vazao_hist', 'oxigenio']])
    else:
        print("⚠️ Nenhum dado encontrado neste boletim.")

# Consolida todos os meses
df = pd.DataFrame(all_rows)
print("\n==============================")
print("✅ Total de linhas extraídas:", len(df))
print("✅ Códigos detectados:", sorted(df['codigo_estacao'].unique()))
print("==============================")



📄 abr_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31  154.0       55.57  10.91        7.13     None
1           EF32  122.4       36.33   2.23        1.83     None
2           EF30   47.5       54.27  65.86       64.99     None
3           EF06   61.8       48.25  77.02       81.95     None
4           EF33   55.6       51.55   8.13        7.44     None



📄 ago_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31    1.0        37.0   9.91        8.32     None
1           EF32    4.0        29.0   1.43        1.73     None
2           EF30    1.0        37.0  20.34       30.88     None
3           EF06    4.0        34.0  25.37       36.13     None
4           EF33   12.0        37.0   3.21        4.65     None



📄 fev_2025.pdf — 4 registros extraídos
  codigo_estacao  chuva  chuva_hist   vazao  vazao_hist oxigenio
0           EF31  159.0       180.0   12.89       11.28     None
1           EF32  109.0       131.0    2.35        2.76     None
2           EF30   95.0       150.0  144.18      116.65     None
3           EF33  152.0       153.0   19.45       14.53     None



📄 jan_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist   vazao  vazao_hist  oxigenio
0           EF31  110.0       181.0    9.62       11.15       NaN
1           EF32  193.0       186.0    2.16        3.13       NaN
2           EF30  161.0       219.0  117.14      131.94       NaN
3           EF06  211.0       178.0  151.39      170.67      0.47
4           EF33  181.0       217.0   13.92       14.60       NaN



📄 jul_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31   22.0        19.0  10.51        7.77     None
1           EF32   25.0        17.0   1.42        1.71     None
2           EF30    9.0        16.0  24.27       33.84     None
3           EF06   17.0        15.0  31.08       39.60     None
4           EF33   27.0        29.0   3.89        5.41     None



📄 jun_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31   57.0        44.0  10.27        8.24     None
1           EF32   29.0        30.0   1.36        1.83     None
2           EF30   41.0        33.0  32.61       57.10     None
3           EF06   58.0        35.0  40.07       67.88     None
4           EF33   69.0        50.0   5.03        7.35     None



📄 mai_2025.pdf — 4 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31   40.6       47.42  10.19        7.38     None
1           EF32   22.4       44.57   1.49        1.89     None
2           EF06   31.8       49.87  44.62       65.56     None
3           EF33   24.0       57.42   4.42        6.02     None



📄 mar_2025.pdf — 3 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31   99.6      136.41   8.41       10.29     None
1           EF32   86.0      115.93   1.51        2.54     None
2           EF33   52.2      150.55   6.95       14.82     None

📄 set_2025.pdf — 5 registros extraídos
  codigo_estacao  chuva  chuva_hist  vazao  vazao_hist oxigenio
0           EF31   72.0        53.0  10.90        8.41     None
1           EF32   35.0        50.0   1.77        2.04     None
2           EF30   15.0        47.0  22.86       30.18     None
3           EF06   12.0        42.0  27.75       35.98     None
4           EF33   30.0        63.0   3.74        5.10     None

✅ Total de linhas extraídas: 41
✅ Códigos detectados: ['EF06', 'EF30', 'EF31', 'EF32', 'EF33']


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# ============================================
# 🧩 1️⃣ Lê a base confiável (valores atuais + oxigênio)
# ============================================

PATH_BASE_ATUAIS = f"{OUT_DIR}/df_resumo_normalizado.csv"

df_base = pd.read_csv(PATH_BASE_ATUAIS, sep=';')
print("📄 Base confiável carregada corretamente:")
print(df_base.head())


# ============================================
# 🧩 2️⃣ Adiciona o código da estação (EFxx) com base no nome do rio
# ============================================

mapa_codigos = {
    'Rio Atibaia': 'EF31',
    'Rio Jaguari': 'EF32',
    'Rio Piracicaba em Monte Alegre': 'EF30',
    'Rio Piracicaba em Artemis': 'EF06',
    'Rio Jundiaí': 'EF33'
}

df_base['codigo_estacao'] = df_base['rio'].map(mapa_codigos)


# ============================================
# 🧩 3️⃣ Junta as duas bases (CSV confiável + PDF extraído)
# ============================================

df_merged = df_base.merge(
    df[['mes_ano', 'codigo_estacao', 'chuva_hist', 'vazao_hist']],
    on=['mes_ano', 'codigo_estacao'],
    how='left'
)

print("\n✅ Dataset mesclado (antes da normalização):")
print(df_merged.head())


# ============================================
# 🧩 4️⃣ Aplica MinMax global em chuva e vazão separadamente
# ============================================

df_final = df_merged.copy()

# --- Normaliza chuva + chuva_hist (0–1 global) ---
scaler_chuva = MinMaxScaler()
df_final[['chuva_norm', 'chuva_hist_norm']] = scaler_chuva.fit_transform(
    df_final[['chuva', 'chuva_hist']]
)

# --- Normaliza vazao + vazao_hist (0–1 global) ---
scaler_vazao = MinMaxScaler()
df_final[['vazao_norm', 'vazao_hist_norm']] = scaler_vazao.fit_transform(
    df_final[['vazao', 'vazao_hist']]
)

# Mantém oxigênio e normalização original da base confiável
df_final['oxigenio'] = df_merged['oxigenio']
df_final['oxigenio_norm'] = df_merged['oxigenio_norm']

print("\n✅ Dataset completo com colunas originais + normalizadas:")
print(df_final[['mes_ano', 'rio', 'codigo_estacao',
                'chuva', 'chuva_norm',
                'chuva_hist', 'chuva_hist_norm',
                'vazao', 'vazao_norm',
                'vazao_hist', 'vazao_hist_norm',
                'oxigenio', 'oxigenio_norm']].head())


# ============================================
# 🧩 5️⃣ (opcional) verifica faixas
# ============================================

print("\n📊 Faixas de normalização:")
print("Chuva →", df_final[['chuva_norm', 'chuva_hist_norm']].min().min(), "a", df_final[['chuva_norm', 'chuva_hist_norm']].max().max())
print("Vazão →", df_final[['vazao_norm', 'vazao_hist_norm']].min().min(), "a", df_final[['vazao_norm', 'vazao_hist_norm']].max().max())


📄 Base confiável carregada corretamente:
   mes_ano                             rio  chuva   vazao  oxigenio  \
0  2025-01       Rio Piracicaba em Artemis  211.0  151.39     96.96   
1  2025-01  Rio Piracicaba em Monte Alegre  161.0  117.14      4.78   
2  2025-01                     Rio Atibaia  110.0    9.62      0.00   
3  2025-01                     Rio Jaguari  193.0    2.16      0.67   
4  2025-01                     Rio Jundiaí  181.0   13.92     44.14   

   chuva_norm  vazao_norm  oxigenio_norm  
0       1.000       0.950          0.970  
1       0.762       0.733          0.048  
2       0.519       0.052          0.000  
3       0.914       0.005          0.007  
4       0.857       0.080          0.441  

✅ Dataset mesclado (antes da normalização):
   mes_ano                             rio  chuva   vazao  oxigenio  \
0  2025-01       Rio Piracicaba em Artemis  211.0  151.39     96.96   
1  2025-01  Rio Piracicaba em Monte Alegre  161.0  117.14      4.78   
2  2025-01      

In [ ]:
# Caminho de saída dentro da pasta resultados
OUT_PATH = f"{OUT_DIR}/df_completo_normalizado.csv"

# Salva como CSV (mantendo separador ";")
df_final.to_csv(OUT_PATH, sep=';', index=False)

print(f"✅ Arquivo salvo com sucesso em:\n{OUT_PATH}")

✅ Arquivo salvo com sucesso em:
/content/drive/MyDrive/pcj_analise/resultados/df_completo_normalizado.csv
